# Modifying the Fibonacci program via a signature file to be more pythonic

This is an example which creates a `.pyf` file from the given fortran file using the numpy.f2py module. Then it modifies the `.pyf` file and compiles the program

Lucas Myers

Written: July 15, 2020

### Generate `.pyf` file corresponding to Fibonacci program

In [1]:
import numpy.f2py

args = ['fib1.f', '-m', 'fib2', '-h', 'fib1.pyf']
failure = numpy.f2py.run_main(args)
print(failure)

Reading fortran codes...
	Reading file 'fib1.f' (format:fix,strict)
Post-processing...
	Block: fib2
			Block: fib
Post-processing (stage 2)...
Saving signatures to file ".\fib1.pyf"
None


### Read in the `.pyf` file

In [2]:
with open('fib1.pyf') as file:
    sig_file = file.read()
    
print(sig_file)

!    -*- f90 -*-
! Note: the context of this file is case sensitive.

python module fib2 ! in 
    interface  ! in :fib2
        subroutine fib(a,n) ! in :fib2:fib1.f
            real*8 dimension(n) :: a
            integer, optional,check(len(a)>=n),depend(a) :: n=len(a)
        end subroutine fib
    end interface 
end python module fib2

! This file was auto-generated with f2py (version:2).
! See http://cens.ioc.ee/projects/f2py2e/



### Modify the `.pyf` file, save it as `fib2.pyf`

In [3]:
modified_sig_file = \
"""!    -*- f90 -*-
python module fib2 
    interface
        subroutine fib(a,n)
            real*8 dimension(n),intent(out),depend(n) :: a
            integer intent(in) :: n
        end subroutine fib
    end interface 
end python module fib2
"""

In [4]:
with open('fib2.pyf', 'w+') as file:
    file.write(modified_sig_file)

### Now read in the source from the original Fibonacci file and compile the program (note that the module name must match the name specified by `python module` in the `.pyf` signature file.

In [5]:
with open('fib1.f') as file:
    source = file.read()
    
args = ['fib2.pyf', '--compiler=mingw32']
module = 'fib2'
failure = numpy.f2py.compile(source, modulename=module, extra_args=args, verbose=0)

print(failure)

0


### Finally, import the package and see if the whole process worked

In [6]:
import fib2
a = fib2.fib(8)
print(a)

[ 0.  1.  1.  2.  3.  5.  8. 13.]
